In [77]:
import pandas as pd

books = pd.read_csv('books_cleaned.csv')

In [78]:
books['categories'].value_counts()

categories
Fiction                       2111
Juvenile Fiction               390
Biography & Autobiography      311
History                        207
Literary Criticism             124
                              ... 
Aged women                       1
Imperialism                      1
Human-animal relationships       1
Amish                            1
Human cloning                    1
Name: count, Length: 479, dtype: int64

In [80]:
#we have to manny categories, lets create two categories : fiction and nonfiction
#for books having >=50 books
categories_mapping = {
    'Fiction':'Fiction',
    'Juvenile Nonfiction':"Children's nonfiction",
    'Juvenile Fiction':"Children's Fiction",
    'Literary Criticism':'Fiction',
    'Philosophy':'Fiction',
    'Religion':'Fiction',
    'History':'Fiction',
    'Biography & Autobiography':'Fiction',
    'Comics & Graphic Novels':'Fiction',
    'Drama':'Fiction',
    'Science':'Nonfiction',
    'Poetry':'Fiction'
}

In [81]:
books['simple_categories'] = books['categories'].map(categories_mapping)

In [8]:
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,NaN
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,NaN
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,NaN
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,NaN
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,NaN
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Fiction


In [9]:
#flter out nan from simple_categories and will use LLM model
books.shape

(5197, 14)

In [82]:
book = books.copy()

In [11]:
books = books[~books['simple_categories'].isna()]

In [14]:
#now we have 3743 books for our LLM model, here we are using only books categories having >=50
books.shape

(3743, 14)

In [61]:
from transformers import pipeline
fiction_categories = ['Fiction', 'Nonfiction']

pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=-1) #-1using CPU , for cuda =0

Device set to use cpu


In [62]:
#access the model

sequence = books.loc[books['simple_categories']=='Fiction', 'description'].reset_index(drop=True)[0]
sequence

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [28]:
pipe(sequence,fiction_categories )

{'sequence': 'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst

In [63]:
import numpy as np
max_index = np.argmax(pipe(sequence,fiction_categories )['scores'])
max_label = pipe(sequence,fiction_categories )['labels'][max_index]
max_label

'Fiction'

In [64]:
#create a prediction function

def generate_predictions(sequence, categories):
    predictions = pipe(sequence,fiction_categories)
    max_index = np.argmax(predictions['scores'])
    max_label = predictions['labels'][max_index]
    return max_label


In [65]:
from tqdm import tqdm
actual_categories = []
predict_categories = []

for i in tqdm(range(0,30)):
    sequence = books.loc[books['simple_categories']=='Fiction', 'description'].reset_index(drop=True)[i]
    predict_categories += [generate_predictions(sequence, fiction_categories)]
    actual_categories += ['Fiction']



100%|██████████| 30/30 [02:59<00:00,  6.00s/it]


In [66]:
for i in tqdm(range(0,30)):
    sequence = books.loc[books['simple_categories']=='Nonfiction', 'description'].reset_index(drop=True)[i]
    predict_categories += [generate_predictions(sequence, fiction_categories)]
    actual_categories += ['Nonfiction']

100%|██████████| 30/30 [02:33<00:00,  5.11s/it]


In [67]:
pred_df = pd.DataFrame({'actual_category':actual_categories,'Pre_category':predict_categories})

In [68]:
pred_df['correct_pred'] = np.where(pred_df['actual_category'] ==pred_df['Pre_category'],1,0 )

In [69]:
#accuracy 80%
pred_df['correct_pred'].sum()/len(pred_df)

np.float64(0.8)

In [93]:
#will check missing categories

isbn_13 = []
predicted_category = []

missing_cats = book.loc[book['simple_categories'].isna(), ['isbn13','description']].reset_index(drop=True)

In [100]:
#crete predict model on missing categories
for i in tqdm(range(0,len(missing_cats))):
    sequence = missing_cats['description'][i]
    predicted_category += [generate_predictions(sequence, fiction_categories)]
    isbn_13 += [missing_cats['isbn13'][i]]


100%|██████████| 1454/1454 [1:42:11<00:00,  4.22s/it]


In [112]:
missing_label_df = pd.DataFrame({'isbn_13':isbn_13,'pred_cats_missing':predicted_category})

In [113]:
missing_label_df = missing_label_df.rename(columns={'isbn_13':'isbn13'})
missing_label_df

,isbn13,pred_cats_missing
0,9780002261982,Fiction
1,9780006280897,Nonfiction
2,9780006280934,Nonfiction
3,9780006380832,Nonfiction
4,9780006470229,Fiction
...,...,...
1449,9788125026600,Nonfiction
1450,9788171565641,Fiction
1451,9788172235222,Fiction
1452,9788173031014,Nonfiction


In [114]:
book = pd.merge(book, missing_label_df, on='isbn13', how='left')

In [118]:
book['simple_categories'] = np.where(book['simple_categories'].isna(),book['pred_cats_missing'],book['simple_categories'])

In [120]:
book = book.drop(columns=['pred_cats_missing'])

In [121]:
book

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,title_and_subtitle,tagged_description,simple_categories
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,Gilead,9780002005883 A NOVEL THAT READERS and critics...,Fiction
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...,Fiction
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,Rage of angels,"9780006178736 A memorable, mesmerizing heroine...",Fiction
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,The Four Loves,9780006280897 Lewis' work on the nature of lov...,Nonfiction
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le...",Nonfiction
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,Mistaken Identity,9788172235222 On A Train Journey Home To North...,Fiction
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,Journey to the East,9788173031014 This book tells the tale of a ma...,Nonfiction
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...,Fiction
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...,Fiction


In [122]:
book.to_csv('books_with_categories.csv', index=False)

In [123]:
#Sentiment Analysis